In [134]:
import pandas as pd
from duckdb import query
import pandas as pd
import numpy as np
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500
pd.set_option('mode.chained_assignment', None)

In [135]:
word = pd.read_pickle('../../pickles/word.pkl')
pars = pd.read_pickle('../../pickles/pars.pkl')
lexn = pd.read_pickle('../../pickles/lexn.pkl')
pdgm = pd.read_pickle('../../pickles/pdgm.pkl')
book = pd.read_pickle('../../pickles/book.pkl')
chap = pd.read_pickle('../../pickles/chap.pkl')
vers = pd.read_pickle('../../pickles/vers.pkl')